In [10]:
# import data
import os
import pandas as pd
import re

df = pd.read_csv('sample-data.csv')

In [34]:
# quick look at data
print df.head()

   id                                        description
0   1  Active classic boxers - There's a reason why o...
1   2  Active sport boxer briefs - Skinning up Glory ...
2   3  Active sport briefs - These superbreathable no...
3   4  Alpine guide pants - Skin in, climb ice, switc...
4   5  Alpine wind jkt - On high ridges, steep ice an...


In [143]:
# remove special characters, numbers, and spaces
def remove_characters(sentence):
    new_sentence = re.sub("</?\w*>|[\W]|[\d]",' ',sentence)
    new_sentence = ' '.join(new_sentence.split())
    return new_sentence

In [139]:
# split description into multiple columns
def split_description(string):
    # name
    string_split = string.split(' - ',1)
    name = string_split[0]
    name = remove_characters(name)
    
    # description
    string_split = string_split[1].split('<b>Details:')
    descrip = string_split[0]
    descrip = remove_characters(descrip)
    
    # details and fabric
    string_split = string_split[1].split('<b>Fabric:')
    details = string_split[0]
    fabric = string_split[1]
    details = remove_characters(details)
    fabric = remove_characters(fabric)
    
    return name, descrip, details, fabric

In [144]:
# new dataframe with new columns
foo = lambda x: pd.Series([i for i in split_description(x)])
df_new = df.loc[:,'description'].apply(foo)
df_new.rename(columns = {0: 'name', 1: 'descrip', 2: 'details', 3: 'fabric'}, inplace = True)
df_new['id'] = df['id']

In [145]:
print df_new.head()

                        name  \
0      Active classic boxers   
1  Active sport boxer briefs   
2        Active sport briefs   
3         Alpine guide pants   
4            Alpine wind jkt   

                                             descrip  \
0  There s a reason why our boxers are a cult fav...   
1  Skinning up Glory requires enough movement wit...   
2  These superbreathable no fly briefs are the mi...   
3  Skin in climb ice switch to rock traverse a kn...   
4  On high ridges steep ice and anything alpine t...   

                                             details  \
0  Silky Capilene fabric is ultralight breathable...   
1  Stretch mesh provides support open weave mesh ...   
2  Stretch mesh provides support open weave mesh ...   
3  Durable stretch woven polyester recycled with ...   
4  Lightweight breathable polyester ripstop fabri...   

                                              fabric  id  
0  oz all recycled polyester with Gladiodor natur...   1  
1  oz polyester

In [147]:
# use only name to cluster
from sklearn.feature_extraction.text import TfidfVectorizer
#define vectorizer parameters

tfidf_vectorizer = TfidfVectorizer(use_idf=True,
                                   stop_words = 'english',
                                   ngram_range=(2,4))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(df_new['name'])

print(tfidf_matrix.shape)

CPU times: user 8.62 ms, sys: 2.43 ms, total: 11.1 ms
Wall time: 9.09 ms
(500, 763)
